In [15]:
import dotenv
import tokenization
from dotenv import load_dotenv
from google import genai
import os

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

sentence = "What is the meaning of life?"*2000
result = client.models.embed_content(
        model="text-embedding-004",
        contents=sentence)

sentence2 = "What is the meaning of life?"*2001
result2 = client.models.embed_content(
    model="text-embedding-004",
    contents=sentence2
)# it truncates? yes it does
print(result2.embeddings[0].values == result.embeddings[0].values)

print(result.embeddings[0].values)

True
[-0.012645576, 0.025118347, -0.014379711, 0.0050726295, -0.023653338, 0.031938877, -9.1477224e-05, 0.04287685, 0.021387381, 0.0060547413, 0.0010401603, 0.007141752, 0.09306691, -0.03321236, 0.034151528, -0.036276523, -0.011947743, 0.019669937, -0.055994406, -0.053298995, 0.025323665, -0.012418596, 0.028655348, -0.009813419, -0.012196734, 0.041764267, 0.018224543, -0.013283424, 0.04033355, 0.022284416, 0.04540627, 0.021591477, 0.009695383, -0.030428367, 0.02607952, -0.006767525, -0.008645974, 0.045967434, -0.024960708, -0.01388307, -0.083812065, -0.0007555806, -0.037535626, 0.05112604, -0.036788467, -0.057376474, -0.015523872, -0.0071095973, -0.00061819644, 0.043835606, -0.022474336, -0.054366574, -0.033500824, -0.0057071913, -0.017879521, -0.050817557, 0.0014600658, -0.019519217, 0.04025256, -0.0065740487, -0.0052175773, -0.039866872, -0.029442703, -0.019547397, -0.025680412, 0.06945987, -0.02622196, -0.023375643, -0.022403382, 0.041072357, -0.035522997, 0.07437623, -0.031329364, 

In [16]:
result = client.models.compute_tokens(
    model="text-embedding-004",
    contents=sentence
)
result

ValueError: This method is only supported in the Vertex AI client.

In [1]:
from src.data_processing.encoder import Encoder
enc = Encoder('voyage-3-large', voyage=True)
tokenizer = enc.tokenizer
example = 'hihi'
encoded = tokenizer.encode(example)
decoded = tokenizer.decode(encoded)
decoded

C:\Users\tiend\PycharmProjects\Fair-Conference-Scheduling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\tiend\PycharmProjects\Fair-Conference-Scheduling\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tiend\.cache\huggingface\hub\models--voyageai--voyage-3-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to

'hihi'

In [4]:
import os
import json
from src import DATA_DIR
from src.utils.io import save_json

with open(os.path.join(DATA_DIR, 'unified_text', 'NeurIPS', 'NeurIPS_2023.json'), 'r', encoding='utf-8') as f:
    data = json.load(f)
for entry in data:
    if entry['id'] == 'kaHpo8OZw2':
        save_json([entry], 'data/emb/NeurIPS/2023/20250603_1057/tmp.json')
        break

In [ ]:
import os
print("Current working directory:", os.getcwd())


In [ ]:
print("API KEY:", os.getenv("OPENAI_API_KEY"))


In [2]:
import pandas as pd
import os
import json
import fitz
from tqdm import tqdm

In [8]:
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")
        return ""

In [16]:
# Read metadata
df = pd.read_csv('./data/metadata/ICLR_2021_2024.csv')

# Create unified text directory if it doesn't exist
os.makedirs('./data/unified_text/ICLR', exist_ok=True)

# Process each year
# years = range(2021, 2025)
years = [2024]
for year in years:
    year_data = []
    year_df = df[df['year'] == year]

    for _, row in tqdm(year_df.iterrows(), total=len(year_df)):
        paper_id = row['id']
        title = row['title']
        txt_path = f'./data/txts/ICLR_2021_2024/{title}.txt'
        pdf_path = f'./data/pdfs/ICLR_2021_2024/{title}.pdf'

        paper_dict = row.to_dict()

        # Try to get text from txt file first
        if os.path.exists(txt_path):
            with open(txt_path, 'r', encoding='utf-8') as f:
                paper_dict['text'] = f.read()
        # If txt doesn't exist, extract from PDF
        elif os.path.exists(pdf_path):
            paper_dict['text'] = extract_text_from_pdf(pdf_path)
        else:
            paper_dict['text'] = ""
            print(f"Neither txt nor pdf found for paper {paper_id}")

        year_data.append(paper_dict)

    # Save to JSON
    output_path = f'./data/unified_text/ICLR/ICLR_{year}.json'
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(year_data, f, ensure_ascii=False, indent=2)


100%|██████████| 86/86 [00:00<00:00, 102.87it/s]


In [15]:
text = extract_text_from_pdf('./data/pdfs/ICLR_2021_2024/An Analytical Solution to Gauss-Newton Loss for Direct Image Alignment.pdf')
with open("./data/unified_text/ICLR/ICLR_2024.json", "r", encoding="utf-8") as f:
    data = json.load(f)
for entry in data:
    if entry['title'] == 'An Analytical Solution to Gauss-Newton Loss for Direct Image Alignment':
        entry['text'] = text
with open("./data/unified_text/ICLR/ICLR_2024.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


In [9]:
# unify for neurips
# Read metadata
df = pd.read_csv('./data/metadata/neurips_2023_2024.csv')

# Create unified text directory if it doesn't exist
os.makedirs('./data/unified_text/NeurIPS', exist_ok=True)

# Process each year
years = range(2023, 2025)
for year in years:
    year_data = []
    year_df = df[df['year'] == year]

    for _, row in tqdm(year_df.iterrows(), total=len(year_df)):
        paper_id = row['id']
        title = row['title']
        txt_path = f'./data/txts/neurips_2023_2024/{title}.txt'
        pdf_path = f'./data/pdfs/neurips_23_24/{title}.pdf'

        paper_dict = row.to_dict()

        # Try to get text from txt file first
        if os.path.exists(txt_path):
            with open(txt_path, 'r', encoding='utf-8') as f:
                paper_dict['text'] = f.read()
        # If txt doesn't exist, extract from PDF
        elif os.path.exists(pdf_path):
            paper_dict['text'] = extract_text_from_pdf(pdf_path)
        else:
            paper_dict['text'] = ""
            print(pdf_path)
            print(f"Neither txt nor pdf found for paper {paper_id}")

        year_data.append(paper_dict)

    # Save to JSON
    output_path = f'./data/unified_text/NeurIPS/{year}.json'
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(year_data, f, ensure_ascii=False, indent=2)


100%|██████████| 72/72 [00:00<00:00, 1984.57it/s]

./data/pdfs/neurips_23_24/The PRISM Alignment Dataset_ What Participatory, Representative and Individualised Human Feedback Reveals About the Subjective and Multicultural Alignment of Large Language Models.pdf
Neither txt nor pdf found for paper DFr5hteojx


In [17]:
text = extract_text_from_pdf('./data/pdfs/neurips_23_24/The PRISM Alignment Dataset_ What Participatory, Representative and Individualised Human Feedback Reveals About the Subjective and Multicultural Alignment of Large L.pdf')
with open("data/unified_text/NeurIPS/NeurIPS_2024.json", "r", encoding="utf-8") as f:
    data = json.load(f)
for entry in data:
    if entry['title'] == 'The PRISM Alignment Dataset_ What Participatory, Representative and Individualised Human Feedback Reveals About the Subjective and Multicultural Alignment of Large Language Models':
        entry['text'] = text
        print('success')
with open("data/unified_text/NeurIPS/NeurIPS_2024.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


success


In [26]:
import pandas as pd
df = pd.read_csv('./data/metadata/ICLR_2021_2024.csv')
df = df[df['year'] == 2024]
print(df.shape)

with open("./data/unified_text/ICLR/ICLR_2024.json", "r", encoding="utf-8") as f:
    data = json.load(f)

count = 0
miss_content = 0
for entry in data:
    count += 1
    if entry['text'] == '':
        miss_content += 1
        print(entry['title'])
print(count)
print(miss_content)

(86, 11)
86
0


In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-7B-instruct", trust_remote_code=True)
# In case you want to reduce the maximum length:
model.max_seq_length = 8192

doc1 = "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day."

doc2 = "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."



doc1_emb = model.encode(doc1)
doc2_emb = model.encode(doc2)

score = (doc1_emb @ doc2_emb.T) * 100


print(score.tolist())


C:\Users\tiend\PycharmProjects\Fair-Conference-Scheduling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\tiend\PycharmProjects\Fair-Conference-Scheduling\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tiend\.cache\huggingface\hub\models--Alibaba-NLP--gte-Qwen2-7B-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer 

KeyboardInterrupt: 

In [ ]:
from src import process_pdfs, DATA_DIR
